In [1]:
import pandas as pd
import numpy as np
import math 
import datetime
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../src')
import utils_explore as ue

In [2]:
data_file = "../data_user/chunk-2019-11.csv"

In [3]:
df = pd.read_csv(data_file, encoding="utf-8")

In [4]:
#cleanup
#rename first columd
df.drop(columns={df.columns[0]: "id" }, inplace = True)
#df.head()

In [5]:
#get records not found in db
not_found = df[df.author.isna()][['bookmaker', 'br-rating', 'event']]
#not_found

In [6]:
# report error/warning codes
df.groupby(by='CODE').agg({'CODE':'count'})

,CODE
CODE,
ERROR,2
LIMIT,36
MISSED,3
NOBOOKIE,4
NOTEXACT,8
SKIP,1
WRONG,1


In [7]:
#BRUTTO results
round(df['return'].sum() - df.stake.sum(), 2)

2.82

In [8]:
# cleanup some data
# 1. wrongly placed bets shouldn't be counted
clean_df = df[df.CODE!="WRONG"]
# 2. apply bets limits, where it (incorrectly) wasn't applied
idx_limit = clean_df.stake > clean_df["MAX stake"]
clean_df.loc[idx_limit, 'return'] = clean_df[idx_limit]["return"] / clean_df[idx_limit].stake * clean_df[idx_limit]["MAX stake"]
clean_df.loc[idx_limit, 'stake'] = clean_df[idx_limit]["MAX stake"]

In [9]:
#NETTO results
netto_win = round(clean_df['return'].sum() - clean_df.stake.sum(), 2)
print("netto WIN", netto_win)

('netto WIN', 122.82)


In [10]:
# report bookmaker
df.groupby(by='bookmaker').agg({'bookmaker':'count'})

,bookmaker
bookmaker,
bet365,27
bwin,24
fonbet,3
one-xgqac,38
tipico,23


In [11]:
#TAXES
german_tax = 0.05
tax_bookies = ['bwin','bet365']
idx_taxable = clean_df.bookmaker.isin(tax_bookies)
total_tax = round(sum(clean_df.loc[idx_taxable, 'return']/(1/german_tax-1)),2)
print("total tax", total_tax)

('total tax', 70.85)


In [12]:
print("total tax %", round(total_tax / (netto_win+total_tax) * 100, 1))

('total tax %', 36.6)


In [13]:
netto_win_taxable = round(clean_df.loc[idx_taxable,'return'].sum() - clean_df.loc[idx_taxable,'stake'].sum(), 2)
print("total tax %, from taxable only", round(total_tax / (netto_win_taxable+total_tax) * 100, 1))

('total tax %, from taxable only', 51.7)


In [14]:
#ODDS
deltaPercent = (clean_df.odds - clean_df.factor)/clean_df.factor * 100
deltaPercent.describe()

count    97.000000
mean     -1.824640
std       9.025815
min     -41.935484
25%      -4.069767
50%      -1.515152
75%       0.000000
max      28.654971
dtype: float64

In [15]:
# stakes
df.groupby(by='stake').agg({'stake':'count'})

,stake
stake,
9.0,1
10.0,32
20.0,41
30.0,31
40.0,7
